<h1> Text Classification using TensorFlow/Keras on Cloud ML Engine </h1>
<h3>Leveraging a pre-trained embedding</h3>

This notebook illustrates:
<ol>
<li> Downloading a pre-trained text embedding
<li> Creating a text classification model using the Estimator API with a Keras model
<li> Training on Cloud ML Engine
<li> Deploying model
<li> Predicting with model
</ol>

In [ ]:
# change these to try this notebook out
BUCKET = 'cloud-training-demos-ml'
PROJECT = 'cloud-training-demos'
REGION = 'us-central1'

In [ ]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.8'

In [ ]:
import tensorflow as tf
print(tf.__version__)

### Pre-requisites
Ensure you have the training files generated from the txtcls_fromscratch notebook. If you don't, go back and run the <a href="txtcls_fromscratch.ipynb">txtcls_fromscratch</a> notebook first.

In [ ]:
!wc -l data/txtcls/*.tsv

### Download Pre-trained Embedding

In previous notebook we trained our word embedding from scratch. Often times we get better performance and/or converge faster by leveraging a pre-trained embedding. This is a similar concept to transfer learning during image classification.

We will use the popular GloVe embedding which is trained on Wikipedia as well as various news sources like the New York Times.

You can read more about Glove at the project homepage: https://nlp.stanford.edu/projects/glove/

*Note: The emedding file is about 900MB so the following cell may take several minutes to run depending on your download speed*

In [ ]:
%bash
wget http://nlp.stanford.edu/data/glove.6B.zip
unzip glove.6B.zip -d data/txtcls/

### TensorFlow/Keras Code

Please explore the code in this <a href="txtclsmodel/trainer">directory</a>: `model.py` contains the TensorFlow model and `task.py` parses command line arguments and launches off the training job. 

This is the **same code** as in the previous notebook. The only difference is we invoke it with the `--embedding_path` parameter

### Run Locally
Let's make sure the code compiles by running for a fraction of an epoch locally. Note the new `--embedding_path` parameter

In [ ]:
%bash
OUTDIR=${PWD}/txtcls_trained
rm -rf $OUTDIR
gcloud ml-engine local train \
   --module-name=trainer.task \
   --package-path=${PWD}/txtclsmodel/trainer \
   -- \
   --output_dir=$OUTDIR \
   --train_data_path=${PWD}/data/txtcls/train.tsv \
   --eval_data_path=${PWD}/data/txtcls/eval.tsv \
   --embedding_path=${PWD}/data/txtcls/glove.6B.200d.txt \
   --num_epochs=0.1

### Train on the Cloud

Let's first copy our embedding file to the cloud:

In [ ]:
!gsutil cp data/txtcls/glove.6B.200d.txt gs://$BUCKET/txtcls/

In [ ]:
%%bash
OUTDIR=gs://${BUCKET}/txtcls/trained_finetune
JOBNAME=txtcls_$(date -u +%y%m%d_%H%M%S)
REGION=us-central1
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
 --region=$REGION \
 --module-name=trainer.task \
 --package-path=${PWD}/txtclsmodel/trainer \
 --job-dir=$OUTDIR \
 --scale-tier=BASIC_GPU \
 --runtime-version=$TFVERSION \
 -- \
 --output_dir=$OUTDIR \
 --train_data_path=gs://${BUCKET}/txtcls/train.tsv \
 --eval_data_path=gs://${BUCKET}/txtcls/eval.tsv \
 --embedding_path=gs://${BUCKET}/txtcls/glove.6B.200d.txt \
 --num_epochs=10

### Monitor training with TensorBoard
If tensorboard appears blank try refreshing after 10 minutes

In [ ]:
from google.datalab.ml import TensorBoard
TensorBoard().start('gs://{}/txtcls/trained_finetune'.format(BUCKET))

In [ ]:
for pid in TensorBoard.list()['pid']:
  TensorBoard().stop(pid)
  print 'Stopped TensorBoard with pid {}'.format(pid)

### Results
What accuracy did you get? Was it an improvement over training the embedding from scratch? 

While the final accuracy may not change significantly, you should notice the model was able to converge to it more quickly given the pre-trained embedding.

### Deploy trained model 

Once your training completes you will see your exported models in the output directory specified in Google Cloud Storage. 

You should see one model for each training checkpoint (default is every 1000 steps).

In [ ]:
%bash
gsutil ls gs://${BUCKET}/txtcls/trained_finetune/export/exporter/

In [ ]:
%%bash
MODEL_NAME="txtcls"
MODEL_VERSION="v1_finetune"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/txtcls/trained_finetune/export/exporter/ | tail -1)
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME} --quiet
#gcloud ml-engine models delete ${MODEL_NAME}
#gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version=$TFVERSION

### Get Predictions

Here are some actual hacker news headlines gathered from July 2018. These titles were not part of the training or evaluation datasets.

In [ ]:
techcrunch=[
  'Uber shuts down self-driving trucks unit',
  'Grover raises €37M Series A to offer latest tech products as a subscription',
  'Tech companies can now bid on the Pentagon’s $10B cloud contract'
]
nytimes=[
  '‘Lopping,’ ‘Tips’ and the ‘Z-List’: Bias Lawsuit Explores Harvard’s Admissions',
  'A $3B Plan to Turn Hoover Dam into a Giant Battery',
  'A MeToo Reckoning in China’s Workplace Amid Wave of Accusations'
]
github=[
  'Show HN: Moon – 3kb JavaScript UI compiler',
  'Show HN: Hello, a CLI tool for managing social media',
  'Firefox Nightly added support for time-travel debugging'
]

Our serving input function expects the already tokenized representations of the headlines, so we do that pre-processing in the code before calling the REST API.

In [ ]:
import pickle
from tensorflow.python.keras.preprocessing import sequence
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

requests = techcrunch+nytimes+github

# Tokenize and pad sentences using same mapping used in the deployed model
tokenizer = pickle.load( open( "txtclsmodel/tokenizer.pickled", "rb" ) )

requests_tokenized = tokenizer.texts_to_sequences(requests)
requests_tokenized = sequence.pad_sequences(requests_tokenized,maxlen=50)

# JSON format the requests
request_data = {'instances':requests_tokenized.tolist()}

# Authenticate and call CMLE prediction API 
credentials = GoogleCredentials.get_application_default()
api = discovery.build('ml', 'v1', credentials=credentials,
            discoveryServiceUrl='https://storage.googleapis.com/cloud-ml/discovery/ml_v1_discovery.json')

parent = 'projects/%s/models/%s/versions/%s' % (PROJECT, 'txtcls', 'v1_finetune')
response = api.projects().predict(body=request_data, name=parent).execute()

# Format and print response
for i in xrange(len(requests)):
  print('\n{}'.format(requests[i]))
  print(' github    : {}'.format(response['predictions'][i]['dense_1'][0]))
  print(' nytimes   : {}'.format(response['predictions'][i]['dense_1'][1]))
  print(' techcrunch: {}'.format(response['predictions'][i]['dense_1'][2]))

### Bonus: Native Tensorflow Predictions


#### Why Native?

Up until now we've been using python functions to do our data pre-processing. This is fine during training, but during serving it adds the limitation that we need a python client in the prediction pipeline.

This limits your serving flexibility. For example, lets say you want to be able to serve this model locally (offline) on a mobile phone. How would you do it? It's non trivial to execute python code on Android. Plus if your tokenization vocabulary ever changed you'd need to update all the clients or else they would be out of sync with the model.

A better way would be to have all of our serving pre-processing to be done using native Tensorflow operations. This way we can take advantage of Tensorflow's hardware agnostic execution engine, and leverage the engineering efforts the Tensorflow team put into making sure our code works whether we're running on a server, mobile, or an embedded device!

### TensorFlow/Keras Code (Native)

Please explore the code in this <a href="txtclsmodel/trainer">directory</a>: `model_native.py` contains the TensorFlow model and `task.py` parses command line arguments and launches off the training job. 

In particular look for the follwing:

1. [tf.keras.preprocessing.text.Tokenizer.fit_on_texts()](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer#fit_on_texts) to generate a mapping from our word vocabulary to integers
2. [tf.gfile](https://www.tensorflow.org/api_docs/python/tf/gfile/GFile) to write the vocabulary mapping to disk
3. [tf.contrib.lookup.index_table_from_file()](https://www.tensorflow.org/api_docs/python/tf/contrib/lookup/index_table_from_file) to encode our sentences into a tensor of their respective word-integers, based on the vocabulary mapping written to disk in the previous step
4. [tf.pad](https://www.tensorflow.org/api_docs/python/tf/pad) to pad all sequences to be the same length

### Train on the Cloud (Native)

Note the new `--native` parameter. This tells `task.py` to call `model_native.py`

In [ ]:
%%bash
OUTDIR=gs://${BUCKET}/txtcls/trained_finetune_native
JOBNAME=txtcls_$(date -u +%y%m%d_%H%M%S)
REGION=us-central1
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
 --region=$REGION \
 --module-name=trainer.task \
 --package-path=${PWD}/txtclsmodel/trainer \
 --job-dir=$OUTDIR \
 --scale-tier=BASIC_GPU \
 --runtime-version=$TFVERSION \
 -- \
 --output_dir=$OUTDIR \
 --train_data_path=gs://${BUCKET}/txtcls/train.tsv \
 --eval_data_path=gs://${BUCKET}/txtcls/eval.tsv \
 --embedding_path=gs://${BUCKET}/txtcls/glove.6B.200d.txt \
 --native \
 --num_epochs=10

### Deploy trained model (Native)

In [ ]:
%%bash
MODEL_NAME="txtcls"
MODEL_VERSION="v1_finetune_native"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/txtcls/trained_finetune_native/export/exporter/ | tail -1)
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME} --quiet
#gcloud ml-engine models delete ${MODEL_NAME}
#gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version=$TFVERSION

### Get Predictions (Native)

Note how we can now feed the titles directly to the model! The text tokenization is done for us inside of the Tensorflow serving_input_fn. 

In [ ]:
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

# JSON format the requests
requests = (techcrunch+nytimes+github)
requests = [x.lower() for x in requests] 
request_data = {'instances': requests}

# Authenticate and call CMLE prediction API 
credentials = GoogleCredentials.get_application_default()
api = discovery.build('ml', 'v1', credentials=credentials,
            discoveryServiceUrl='https://storage.googleapis.com/cloud-ml/discovery/ml_v1_discovery.json')

parent = 'projects/%s/models/%s/versions/%s' % (PROJECT, 'txtcls', 'v1_finetune_native')
response = api.projects().predict(body=request_data, name=parent).execute()

# Format and print response
for i in xrange(len(requests)):
  print('\n{}'.format(requests[i]))
  print(' github    : {}'.format(response['predictions'][i]['dense_1'][0]))
  print(' nytimes   : {}'.format(response['predictions'][i]['dense_1'][1]))
  print(' techcrunch: {}'.format(response['predictions'][i]['dense_1'][2]))

#### References
- This implementation is based on code from: https://github.com/google/eng-edu/tree/master/ml/guides/text_classification.
- See the full text classification tutorial at: https://developers.google.com/machine-learning/guides/text-classification/

Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License